<a href="https://colab.research.google.com/github/alexshultz/comfyui-colab/blob/main/comfyui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Git clone the repo and install the requirements. (ignore the pip errors about protobuf)

In [ ]:
#@title Connect Google Drive


OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
WORKSPACE = 'ComfyUI'
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive


Mounting Google Drive...
/
Mounted at /content/drive
/content/drive/MyDrive


In [ ]:
#@title Environment Setup


OPTIONS = {}

USE_GOOGLE_DRIVE = False  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
WORKSPACE = 'ComfyUI'
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !git pull

!echo -= Install dependencies =-
# Install xFormers for CUDA 12.4 first
!pip install xformers!=0.0.18 --index-url https://download.pytorch.org/whl/cu124
# Then install remaining requirements
!pip install -r requirements.txt

# InsightFace and onnxruntime
!echo -= Installing InsightFace and onnxruntime for IP-Adapter FaceID =-
!pip install insightface onnxruntime-gpu

# Update or install IP-Adapter custom nodes
!echo -= Updating ComfyUI_IPAdapter_plus =-
# If directory exists and is a git repo, update it; otherwise, clone fresh
!cd /content/drive/MyDrive/ComfyUI/custom_nodes && (cd ComfyUI_IPAdapter_plus && git pull || git clone https://github.com/cubiq/ComfyUI_IPAdapter_plus.git ComfyUI_IPAdapter_plus)
# Install dependencies
!if [ -f /content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI_IPAdapter_plus/requirements.txt ]; then pip install -r /content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI_IPAdapter_plus/requirements.txt; else echo "requirements.txt missing, installing manually" && pip install insightface opencv-python-headless numpy torch torchvision; fi

# Update or install ComfyUI_essentials custom nodes
!echo -= Updating ComfyUI_essentials =-
# If directory exists and is a git repo, update it; otherwise, clone fresh
!cd /content/drive/MyDrive/ComfyUI/custom_nodes && (cd ComfyUI_essentials && git pull || git clone https://github.com/cubiq/ComfyUI_essentials.git ComfyUI_essentials)
# Install dependencies
!if [ -f /content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI_essentials/requirements.txt ]; then pip install -r /content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI_essentials/requirements.txt; else echo "No additional requirements for ComfyUI_essentials"; fi


# Update or install Comfy_Dungeon custom nodes
# https://github.com/cubiq/Comfy_Dungeon
!echo -= Updating Comfy_Dungeon =-
!cd /content/drive/MyDrive/ComfyUI/custom_nodes && (cd Comfy_Dungeon && git pull || git clone https://github.com/cubiq/Comfy_Dungeon.git Comfy_Dungeon)
# No requirements.txt specified in README, assuming base ComfyUI deps suffice

# Install huggingface model dowloader
!echo -= huggingface_hub =-
!pip install huggingface_hub


!echo -= Done =-


Download some models/checkpoints/vae or custom comfyui nodes (uncomment the commands for the ones you want)

Great tools in the AP-Adapter package
`https://github.com/cubiq/ComfyUI_IPAdapter_plus?tab=readme-ov-file`

In [ ]:
#@title Model Downloads

import os
import shutil
import time
from huggingface_hub import snapshot_download
from google.colab import userdata
from google.colab import auth
from google.colab import drive
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import google.auth
from google.auth.transport.requests import Request

# Check if Google Drive is already mounted
def is_drive_mounted():
    return os.path.isdir('/content/drive')

# Mount Drive if not already mounted
if not is_drive_mounted():
    print("Mounting Google Drive...")
    drive.mount('/content/drive')
else:
    print("Google Drive already mounted.")

# Check if Drive API credentials are already authenticated
def is_drive_api_authenticated():
    creds, _ = google.auth.default()
    if creds and creds.valid:
        return True
    elif creds and creds.expired and creds.refresh_token:
        try:
            creds.refresh(Request())
            return True
        except Exception:
            return False
    return False

# Initialize Google Drive API with explicit authentication if needed
drive_service = None
if is_drive_api_authenticated():
    print("Google Drive API already authenticated from previous block.")
    drive_service = build('drive', 'v3')
else:
    print("Authenticating Google Drive API...")
    auth.authenticate_user()
    try:
        drive_service = build('drive', 'v3')
        drive_service.files().get(fileId='root').execute()
        print("Google Drive API authenticated successfully.")
    except HttpError as e:
        print(f"Failed to initialize Drive API: {e}, proceeding without server-side copying.")
        drive_service = None

# Get Hugging Face token from Colab Secrets
HF_TOKEN = userdata.get('HF_TOKEN')  # Add this to Secrets as 'HF_TOKEN'

# Ensure standard ComfyUI directories exist
base_dir = "/content/drive/MyDrive/ComfyUI/models"
!mkdir -p {base_dir}/checkpoints
!mkdir -p {base_dir}/loras
!mkdir -p {base_dir}/ipadapter
!mkdir -p {base_dir}/vae
!mkdir -p {base_dir}/clip_vision
!mkdir -p {base_dir}/controlnet
!mkdir -p {base_dir}/style_models
!mkdir -p {base_dir}/gligen
!mkdir -p {base_dir}/upscale_models
!mkdir -p {base_dir}/insightface
!mkdir -p {base_dir}/hf_downloads

# Helper function to get Drive file ID from path with retry
def get_file_id(drive_service, file_path, retries=3, delay=2):
    for attempt in range(retries):
        try:
            # Adjust path to start from MyDrive (remove /content/drive/)
            parts = file_path.split('/')[3:]  # Skip /content/drive/
            parent_id = 'root'  # Start at Drive root (MyDrive)
            full_path = '/'.join(parts)
            for part in parts:
                if not part:
                    continue
                query = f"'{parent_id}' in parents and name = '{part}' and trashed = false"
                response = drive_service.files().list(q=query, fields='files(id)').execute()
                files = response.get('files', [])
                if not files:
                    print(f"Attempt {attempt + 1}/{retries}: Could not find '{part}' in path {full_path}")
                    return None
                parent_id = files[0]['id']
            return parent_id
        except HttpError as e:
            print(f"Attempt {attempt + 1}/{retries}: Error finding file ID for {file_path}: {e}")
            if attempt < retries - 1:
                time.sleep(delay)
            else:
                return None

# Helper function to copy file server-side using Drive API
def drive_copy_file(drive_service, src_path, dest_path):
    if not drive_service:
        return False
    if not os.path.exists(src_path):
        print(f"Source file {src_path} does not exist locally, cannot copy")
        return False

    src_id = get_file_id(drive_service, src_path)
    if not src_id:
        print(f"Could not find {src_path} in Drive, falling back to shutil")
        return False

    dest_dir = os.path.dirname(dest_path)
    dest_dir_id = get_file_id(drive_service, dest_dir)
    if not dest_dir_id:
        parts = dest_dir.split('/')[3:]  # Skip /content/drive/
        parent_id = 'root'
        for part in parts:
            if not part:
                continue
            query = f"'{parent_id}' in parents and name = '{part}' and trashed = false"
            response = drive_service.files().list(q=query, fields='files(id)').execute()
            files = response.get('files', [])
            if not files:
                folder_metadata = {
                    'name': part,
                    'mimeType': 'application/vnd.google-apps.folder',
                    'parents': [parent_id]
                }
                folder = drive_service.files().create(body=folder_metadata, fields='id').execute()
                parent_id = folder.get('id')
            else:
                parent_id = files[0]['id']
        dest_dir_id = parent_id

    file_metadata = {
        'name': os.path.basename(dest_path),
        'parents': [dest_dir_id]
    }
    try:
        drive_service.files().copy(fileId=src_id, body=file_metadata).execute()
        print(f"Server-side copied {src_path} to {dest_path}")
        return True
    except HttpError as e:
        print(f"Drive API error copying {src_path} to {dest_path}: {e}, falling back to shutil")
        return False

# Sort downloaded files into model-specific subdirectories, preserving subdir structure
def sort_hf_files(src_dir, base_dir, model_name):
    if not model_name:
        model_name = os.path.basename(src_dir)

    for root, _, files in os.walk(src_dir):
        for file in files:
            src_path = os.path.join(root, file)
            src_mtime = os.path.getmtime(src_path)
            src_size = os.path.getsize(src_path)

            if file.endswith('.safetensors'):
                if 'ip-adapter' in file.lower():
                    type_dir = f"{base_dir}/ipadapter"
                elif 'lora' in file.lower():
                    type_dir = f"{base_dir}/loras"
                elif 'vae' in file.lower():
                    type_dir = f"{base_dir}/vae"
                elif 'clip' in file.lower() or 'text_encoder' in file.lower() or 'image_encoder' in file.lower():
                    type_dir = f"{base_dir}/clip_vision"
                elif 'control' in file.lower():
                    type_dir = f"{base_dir}/controlnet"
                elif 'style' in file.lower():
                    type_dir = f"{base_dir}/style_models"
                elif 'gligen' in file.lower():
                    type_dir = f"{base_dir}/gligen"
                else:
                    type_dir = f"{base_dir}/checkpoints"
            elif file.endswith(('.pth', '.ckpt', '.bin', '.pt')):
                if 'vae' in file.lower():
                    type_dir = f"{base_dir}/vae"
                elif 'control' in file.lower():
                    type_dir = f"{base_dir}/controlnet"
                elif 'style' in file.lower():
                    type_dir = f"{base_dir}/style_models"
                elif 'gligen' in file.lower():
                    type_dir = f"{base_dir}/gligen"
                elif 'esrgan' in file.lower() or 'upscale' in file.lower():
                    type_dir = f"{base_dir}/upscale_models"
                else:
                    type_dir = f"{base_dir}/checkpoints"
            else:
                continue

            relative_path = os.path.relpath(src_path, src_dir)
            model_subdir = os.path.join(type_dir, model_name)
            dest_dir = os.path.join(model_subdir, os.path.dirname(relative_path))
            !mkdir -p {dest_dir}
            dest_path = os.path.join(dest_dir, file)

            if not os.path.exists(dest_path):
                if not drive_copy_file(drive_service, src_path, dest_path):
                    shutil.copy2(src_path, dest_path)
                    print(f"Copied {file} to {dest_path} (new file, via shutil)")
            elif os.path.getmtime(dest_path) < src_mtime or os.path.getsize(dest_path) != src_size:
                if not drive_copy_file(drive_service, src_path, dest_path):
                    shutil.copy2(src_path, dest_path)
                    print(f"Copied {file} to {dest_path} (updated file, via shutil)")
            else:
                print(f"Skipped {file} - up-to-date in {dest_path}")

# Download InsightFace buffalo_l models
!echo -= Downloading InsightFace buffalo_l models =-
!wget -c https://github.com/deepinsight/insightface/releases/download/v0.7/buffalo_l.zip -O {base_dir}/insightface/buffalo_l.zip
!unzip -o {base_dir}/insightface/buffalo_l.zip -d {base_dir}/insightface/
!rm {base_dir}/insightface/buffalo_l.zip

# Download models using snapshot_download
!echo -= Downloading Models =-

# Dreamshaper 8 (checkpoint via wget, as it's from Civitai) - Skip if already present
dreamshaper_8_subdir = f"{base_dir}/checkpoints/dreamshaper_8"
dreamshaper_8_path = f"{dreamshaper_8_subdir}/dreamshaper_8.safetensors"
old_dreamshaper_8_path = f"{base_dir}/checkpoints/dreamshaper_8.safetensors"
if not os.path.exists(dreamshaper_8_path) and not os.path.exists(old_dreamshaper_8_path):
    !mkdir -p {dreamshaper_8_subdir}
    !wget -c https://civitai.com/api/download/models/128713 -O {dreamshaper_8_path}
    print(f"Downloaded Dreamshaper 8 to {dreamshaper_8_path}")
else:
    print(f"Skipped Dreamshaper 8 - already exists at {dreamshaper_8_path if os.path.exists(dreamshaper_8_path) else old_dreamshaper_8_path}")

# ProteusV0.3
print("Downloading ProteusV0.3...")
snapshot_download(repo_id="dataautogpt3/ProteusV0.3", local_dir=f"{base_dir}/hf_downloads/ProteusV0.3", allow_patterns=["*.safetensors"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/ProteusV0.3", base_dir, "ProteusV0.3")

# dreamshaper-xl-v2-turbo
print("Downloading dreamshaper-xl-v2-turbo...")
snapshot_download(repo_id="Lykon/dreamshaper-xl-v2-turbo", local_dir=f"{base_dir}/hf_downloads/dreamshaper-xl-v2-turbo", allow_patterns=["*.safetensors"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/dreamshaper-xl-v2-turbo", base_dir, "dreamshaper-xl-v2-turbo")

# lcm-lora-sdxl
print("Downloading lcm-lora-sdxl...")
snapshot_download(repo_id="latent-consistency/lcm-lora-sdxl", local_dir=f"{base_dir}/hf_downloads/lcm-lora-sdxl", allow_patterns=["*.safetensors"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/lcm-lora-sdxl", base_dir, "lcm-lora-sdxl")

# Stable Diffusion XL base 1.0
print("Downloading SD-XL 1.0-base...")
snapshot_download(repo_id="stabilityai/stable-diffusion-xl-base-1.0", local_dir=f"{base_dir}/hf_downloads/SD-XL_1.0-base", allow_patterns=["*.safetensors"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/SD-XL_1.0-base", base_dir, "SD-XL_1.0-base")

# clip_vision_g (SDXL ReVision)
print("Downloading clip_vision_g...")
snapshot_download(repo_id="comfyanonymous/clip_vision_g", local_dir=f"{base_dir}/hf_downloads/clip_vision_g", allow_patterns=["*.safetensors"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/clip_vision_g", base_dir, "clip_vision_g")

# Stable Diffusion v1.5
print("Downloading Stable Diffusion v1.5...")
snapshot_download(repo_id="Comfy-Org/stable-diffusion-v1-5-archive", local_dir=f"{base_dir}/hf_downloads/Stable_Diffusion_v1.5", allow_patterns=["*.safetensors"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/Stable_Diffusion_v1.5", base_dir, "Stable_Diffusion_v1.5")

# OrangeMixs
print("Downloading OrangeMixs...")
snapshot_download(repo_id="WarriorMama777/OrangeMixs", local_dir=f"{base_dir}/hf_downloads/OrangeMixs", allow_patterns=["*.safetensors"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/OrangeMixs", base_dir, "OrangeMixs")

# Anything V3 (gated, using token) - Commented out
# print("Downloading Anything V3...")
# snapshot_download(repo_id="Linaqruf/anything-v3.0", local_dir=f"{base_dir}/hf_downloads/Anything_V3", allow_patterns=["*.safetensors"], ignore_patterns=["*.git*"], token=HF_TOKEN)
# sort_hf_files(f"{base_dir}/hf_downloads/Anything_V3", base_dir, "Anything_V3")

# IP-Adapter
print("Downloading IP-Adapter...")
snapshot_download(repo_id="h94/IP-Adapter", local_dir=f"{base_dir}/hf_downloads/IP-Adapter", allow_patterns=["*.safetensors"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/IP-Adapter", base_dir, "IP-Adapter")

# IP-Adapter-FaceID
print("Downloading IP-Adapter-FaceID...")
snapshot_download(repo_id="h94/IP-Adapter-FaceID", local_dir=f"{base_dir}/hf_downloads/IP-Adapter-FaceID", allow_patterns=["*.safetensors", "*.bin"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/IP-Adapter-FaceID", base_dir, "IP-Adapter-FaceID")

# NSFWmodel
print("Downloading NSFWmodel...")
snapshot_download(repo_id="lexa862/NSFWmodel", local_dir=f"{base_dir}/hf_downloads/NSFWmodel", allow_patterns=["*.safetensors"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/NSFWmodel", base_dir, "NSFWmodel")

# illuminatiDiffusionV1_v11_unCLIP
print("Downloading illuminatiDiffusionV1_v11_unCLIP...")
snapshot_download(repo_id="comfyanonymous/illuminatiDiffusionV1_v11_unCLIP", local_dir=f"{base_dir}/hf_downloads/illuminatiDiffusionV1_v11_unCLIP", allow_patterns=["*.safetensors"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/illuminatiDiffusionV1_v11_unCLIP", base_dir, "illuminatiDiffusionV1_v11_unCLIP")

# wd-1.5-beta2_unCLIP
print("Downloading wd-1.5-beta2_unCLIP...")
snapshot_download(repo_id="comfyanonymous/wd-1.5-beta2_unCLIP", local_dir=f"{base_dir}/hf_downloads/wd-1.5-beta2_unCLIP", allow_patterns=["*.safetensors"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/wd-1.5-beta2_unCLIP", base_dir, "wd-1.5-beta2_unCLIP")

# sd-vae-ft-mse-original
print("Downloading sd-vae-ft-mse-original...")
snapshot_download(repo_id="stabilityai/sd-vae-ft-mse-original", local_dir=f"{base_dir}/hf_downloads/sd-vae-ft-mse-original", allow_patterns=["*.safetensors"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/sd-vae-ft-mse-original", base_dir, "sd-vae-ft-mse-original")

# Waifu Diffusion v1.4
print("Downloading Waifu Diffusion v1.4...")
snapshot_download(repo_id="hakurei/waifu-diffusion-v1-4", local_dir=f"{base_dir}/hf_downloads/Waifu_Diffusion_v1.4", allow_patterns=["*.safetensors", "*.ckpt"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/Waifu_Diffusion_v1.4", base_dir, "Waifu_Diffusion_v1.4")

# clip-vit-large-patch14
print("Downloading clip-vit-large-patch14...")
snapshot_download(repo_id="openai/clip-vit-large-patch14", local_dir=f"{base_dir}/hf_downloads/clip-vit-large-patch14", allow_patterns=["*.bin"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/clip-vit-large-patch14", base_dir, "clip-vit-large-patch14")

# ControlNet-v1-1_fp16_safetensors
print("Downloading ControlNet-v1-1_fp16_safetensors...")
snapshot_download(repo_id="comfyanonymous/ControlNet-v1-1_fp16_safetensors", local_dir=f"{base_dir}/hf_downloads/ControlNet-v1-1_fp16_safetensors", allow_patterns=["*.safetensors"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/ControlNet-v1-1_fp16_safetensors", base_dir, "ControlNet-v1-1_fp16_safetensors")

# Control-LoRA Model
print("Downloading Control-LoRA Model...")
snapshot_download(repo_id="stabilityai/control-lora", local_dir=f"{base_dir}/hf_downloads/Control-LoRA_Model", allow_patterns=["*.safetensors"], ignore_patterns=["*.git*"])
sort_hf_files(f"{base_dir}/hf_downloads/Control-LoRA_Model", base_dir, "Control-LoRA_Model")

# GLIGEN
!echo -= Downloading GLIGEN =-
!wget -c https://huggingface.co/comfyanonymous/GLIGEN_pruned_safetensors/resolve/main/gligen_sd14_textbox_pruned_fp16.safetensors -O {base_dir}/gligen/gligen_sd14_textbox_pruned_fp16.safetensors

# ESRGAN upscale models
!echo -= Downloading ESRGAN upscale models =-
!wget -c https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -O {base_dir}/upscale_models/RealESRGAN_x4plus.pth
!wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x2.pth -O {base_dir}/upscale_models/RealESRGAN_x2.pth
!wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x4.pth -O {base_dir}/upscale_models/RealESRGAN_x4.pth

# Verify downloads
!echo -= Verifying Downloads =-
!ls -lh {base_dir}/checkpoints/
!ls -lh {base_dir}/loras/
!ls -lh {base_dir}/ipadapter/
!ls -lh {base_dir}/vae/
!ls -lh {base_dir}/clip_vision/
!ls -lh {base_dir}/controlnet/
!ls -lh {base_dir}/style_models/
!ls -lh {base_dir}/gligen/
!ls -lh {base_dir}/upscale_models/

!echo -= Done =-

In [ ]:
!pip uninstall civitdl

In [5]:
#@title Direct Model Downloads

from google.colab import userdata
# get hugginface token
userdata.get('HF_TOKEN')
#get civitai token
userdata.get('CIVIT_TOKEN')
OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
WORKSPACE = 'ComfyUI'
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive/ComfyUI

###install downloaders
#civitai
#!echo -= civitai =-
#!pip install civitdl
# Install huggingface model downloader
#!echo -= huggingface_hub =-
#!pip install huggingface_hub

#DreamShaper - A versatile SD 1.5-based model published in July 2023 with over 13,000 positive reviews. It's known for better handling of character LoRAs, improved photorealism, and better NSFW capabilities.
!echo -= Dreamshaper =-
!wget -c https://civitai.com/models/4384/dreamshaper -O ./models/checkpoints/dreamshaper_8.safetensors
!echo -= Dreamshaper XL =-
#DreamShaper XL - The SDXL version published in February 2024 with nearly 5,000 positive reviews. The current version is based on SDXL Turbo and should be used with CFG scale 2 and 4-8 sampling steps, preferably with DPM++ SDE Karras sampler.
!wget -c https://civitai.com/models/112902/dreamshaper-xl -O ./models/checkpoints/dreamshaperXL_v21TurboDPMSDE.safetensors

### SDXL
### I recommend these workflow examples: https://comfyanonymous.github.io/ComfyUI_examples/sdxl/

!wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors -P ./models/checkpoints/
!wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors -P ./models/checkpoints/

# SDXL ReVision
!wget -c https://huggingface.co/comfyanonymous/clip_vision_g/resolve/main/clip_vision_g.safetensors -P ./models/clip_vision/

# SD1.5
!wget -c https://huggingface.co/Comfy-Org/stable-diffusion-v1-5-archive/resolve/main/v1-5-pruned-emaonly-fp16.safetensors -P ./models/checkpoints/

# SD VAE
!wget -c https://huggingface.co/stabilityai/sd-vae-ft-mse/resolve/main/diffusion_pytorch_model.safetensors -P ./vae/checkpoints/sd-vae-ft-mse.safetensors

# Dreamshaper 8 (for SD1.5)
!wget -c https://civitai.com/api/download/models/128713 -O ./models/checkpoints/dreamshaper_8.safetensors

## IP-Adapter
# IP-Adapter Models
!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plus_sd15.bin -P ./models/ipadapter/
# Basic model, average strength
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15.safetensors -P ./models/ipadapter/
# Light impact model
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15_light_v11.bin -P ./models/ipadapter/
# Plus model, very strong
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.safetensors -P ./models/ipadapter/
# Face model, portraits
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus-face_sd15.safetensors -P ./models/ipadapter/
# Stronger face model, not necessarily better
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-full-face_sd15.safetensors -P ./models/ipadapter/
# Base model, requires bigG clip vision encoder
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15_vit-G.safetensors -P ./models/ipadapter/
# SDXL model
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter_sdxl_vit-h.safetensors -P ./models/ipadapter/
# SDXL plus model
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus_sdxl_vit-h.safetensors -P ./models/ipadapter/
# SDXL face model
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus-face_sdxl_vit-h.safetensors -P ./models/ipadapter/
# vit-G SDXL model, requires bigG clip vision encoder
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter_sdxl.safetensors -P ./models/ipadapter/

# IP-Adapter - FaceID models require insightface, you need to install it in your ComfyUI environment. Check this issue for help. Remember that most FaceID models also need a LoRA.
# base FaceID model
!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sd15.bin -P ./models/ipadapter/
# FaceID plus v2
!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15.bin -P ./models/ipadapter/
# text prompt style transfer for portraits
!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-portrait-v11_sd15.bin -P ./models/ipadapter/
# SDXL base FaceID
!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sdxl.bin -P ./models/ipadapter/
# SDXL plus v2
!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sdxl.bin -P ./models/ipadapter/
# SDXL text prompt style transfer
!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-portrait_sdxl.bin -P ./models/ipadapter/
# very strong style transfer SDXL only
!wget -c ip-adapter-faceid-portrait_sdxl_unnorm.bin -P ./models/ipadapter/
# IP-Adapter - community - general composition ignoring style and content, more about it at https://huggingface.co/ostris/ip-composition-adapter
!wget -c https://huggingface.co/ostris/ip-composition-adapter/resolve/main/ip_plus_composition_sd15.safetensors -P ./models/ipadapter/
# SDXL version
!wget -c https://huggingface.co/ostris/ip-composition-adapter/resolve/main/ip_plus_composition_sdxl.safetensors -P ./models/ipadapter/

# IP-Adapter Loras
# IP-Adapter - FaceID - Most FaceID models require a LoRA. If you use the IPAdapter Unified Loader FaceID it will be loaded automatically if you follow the naming convention. Otherwise you have to load them manually, be careful each FaceID model has to be paired with its own specific LoRA.
!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plus_sd15_lora.safetensors -P ./models/loras/
!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15_lora.safetensors -P ./models/loras/
# SDXL FaceID LoRA
!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sdxl_lora.safetensors -P ./models/loras/
# SDXL plus v2 LoRA
!wget -c https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sdxl_lora.safetensors -P ./models/loras/

# IP-Adapter CLIPVision models
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/model.safetensors -O ./models/clip_vision/CLIP-ViT-H-14-laion2B-s32B-b79K.safetensors
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/image_encoder/model.safetensors -O ./models/clip_vision/CLIP-ViT-bigG-14-laion2B-39B-b160k.safetensors


# SD2
#!wget -c https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.safetensors -P ./models/checkpoints/

# Some SD1.5 anime style
#!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_hard.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A1_orangemixs.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A3_orangemixs.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-fp16-pruned.safetensors -P ./models/checkpoints/

# Waifu Diffusion 1.5 (anime style SD2.x 768-v)
#!wget -c https://huggingface.co/waifu-diffusion/wd-1-5-beta3/resolve/main/wd-illusion-fp16.safetensors -P ./models/checkpoints/


# unCLIP models
!wget -c https://huggingface.co/comfyanonymous/illuminatiDiffusionV1_v11_unCLIP/resolve/main/illuminatiDiffusionV1_v11-unclip-h-fp16.safetensors -P ./models/checkpoints/
!wget -c https://huggingface.co/comfyanonymous/wd-1.5-beta2_unCLIP/resolve/main/wd-1-5-beta2-aesthetic-unclip-h-fp16.safetensors -P ./models/checkpoints/


# VAE
!wget -c https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -P ./models/vae/
#!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt -P ./models/vae/
!wget -c https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -P ./models/vae/


# Loras
!wget -c https://civitai.com/api/download/models/10350 -O ./models/loras/theovercomer8sContrastFix_sd21768.safetensors #theovercomer8sContrastFix SD2.x 768-v
!wget -c https://civitai.com/api/download/models/10638 -O ./models/loras/theovercomer8sContrastFix_sd15.safetensors #theovercomer8sContrastFix SD1.x
!wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_offset_example-lora_1.0.safetensors -P ./models/loras/ #SDXL offset noise lora

# T2I-Adapter
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_depth_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_seg_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_sketch_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_keypose_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_openpose_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_canny_sd14v1.pth -P ./models/controlnet/

# T2I Styles Model
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth -P ./models/style_models/

# CLIPVision model (needed for styles model)
!wget -c https://huggingface.co/openai/clip-vit-large-patch14/resolve/main/pytorch_model.bin -O ./models/clip_vision/clip_vit14.bin

# ControlNet
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11u_sd15_tile_fp16.safetensors -P ./models/controlnet/

# ControlNet SDXL
!wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-canny-rank256.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-depth-rank256.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-recolor-rank256.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/stabilityai/control-lora/resolve/main/control-LoRAs-rank256/control-lora-sketch-rank256.safetensors -P ./models/controlnet/

# Controlnet Preprocessor nodes by Fannovel16
!cd custom_nodes && git clone https://github.com/Fannovel16/comfy_controlnet_preprocessors; cd comfy_controlnet_preprocessors && python install.py


# GLIGEN
!wget -c https://huggingface.co/comfyanonymous/GLIGEN_pruned_safetensors/resolve/main/gligen_sd14_textbox_pruned_fp16.safetensors -P ./models/gligen/


# ESRGAN upscale model
!wget -c https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P ./models/upscale_models/
!wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x2.pth -P ./models/upscale_models/
!wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x4.pth -P ./models/upscale_models/


# Download and extract InsightFace buffalo_l models
!echo -= Downloading InsightFace buffalo_l models =-
!wget -c https://github.com/deepinsight/insightface/releases/download/v0.7/buffalo_l.zip -O /content/drive/MyDrive/ComfyUI/models/insightface/buffalo_l.zip
!unzip -o /content/drive/MyDrive/ComfyUI/models/insightface/buffalo_l.zip -d /content/drive/MyDrive/ComfyUI/models/insightface/
!rm /content/drive/MyDrive/ComfyUI/models/insightface/buffalo_l.zip


# Download required SDXL LCM LoRA model (rename as required by Comfy_Dungeon)
!echo -= Downloading SDXL LCM LoRA for Comfy_Dungeon =-
!wget -c https://huggingface.co/latent-consistency/lcm-lora-sdxl/resolve/main/pytorch_lora_weights.safetensors -O /content/drive/MyDrive/ComfyUI/models/loras/lcm-lora-sdxl.safetensors

!echo -= Done =-


Mounting Google Drive...
/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ComfyUI
./models/hf_downloads: Is a directory
./models/hf_downloads: Is a directory
--2025-03-23 02:26:39--  https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors
Resolving huggingface.co (huggingface.co)... 18.172.134.4, 18.172.134.24, 18.172.134.124, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cas-bridge.xethub.hf.co/xet-bridge-us/64bfcd5ff462a99a04fd1ec8/3d6f740fa52572e1071b8ecb7c5f8a8e2cbef596a51121102877bd9900078891?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Content-Sha256=UNSIGNED-PAYLOAD&X-Amz-Credential=cas%2F20250323%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20250323T022350Z&X-Amz-Expires=3600&X-Amz-Signature=1ce5f6b0abf46e1a552464420ca2afeabf2c6af896

### Run ComfyUI with cloudflared (Recommended Way)




In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server